In [1]:
import syft as sy

In [2]:
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [3]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [11]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 95bf45f62ad142149f183e6fb5767b60>,[DO1 data],Dataset of 6 samples,<class 'torch.Tensor'>


In [12]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: e91731b0fa2145ee8588078f1fe14f91>,[DO2 data],Dataset of 5 samples,<class 'torch.Tensor'>


j'ai fait une petite faute de frape entre duet1 et duet2 du coup je v inversser pour l'eviter dans le nommage des data_pointers

In [14]:
data1_pointer = duet2.store[0]
data2_pointer = duet1.store[0]

In [16]:
in_dim = 1 
out_dim = 1

In [34]:
class SyNet(sy.Module):
    def __init__(self,torch_ref): 
        super(SyNet,self ).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim,out_dim)
    def forward(self,x):
        x = self.linear(x)
        return x

In [35]:
def train(iterations,model,torch_ref,optim,data_ptr,target_ptr): 
    losses = []
    for i in range(iterations): 
        optim.zero_grad()
        output = model(data_ptr)
        loss = torch_ref.nn.functional.mse_loss(output,target_ptr)
        loss_item = loss.item()
        loss_value = loss_item.get(
            reason ="To evalute training process ",
            request_block = True,
            timeout_secs = 5
        )
        print("Epoch",i, " loss", loss_value)
        losses.append(loss_value)
        loss.backward()
        optim.step()
    return losses
        

In [36]:
import torch as th 
import numpy as np 
import torch

In [37]:
base_model = SyNet(torch)

Train the first remote model with the data with the first data ouwner

In [38]:
remote_model1= base_model.send(duet2)

In [39]:
remote_torch1 = duet2.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params , lr=0.1)

In [40]:
target1_ptr = th.FloatTensor(np.array([3,4,1,0,3,4]).reshape(-1,1))

In [74]:
target1_ptr

tensor([[3.],
        [4.],
        [1.],
        [0.],
        [3.],
        [4.]])

In [76]:
type(data1_pointer)

syft.proxy.torch.TensorPointer

In [75]:
iterations = 10 
losses = train(iterations,remote_model1,remote_torch1,optim1,data1_pointer,target1_ptr)

Epoch 0  loss 7.522589206695557
Epoch 1  loss 3.769211530685425
Epoch 2  loss 3.5614607334136963
Epoch 3  loss 5.994535446166992
Epoch 4  loss 9.325160026550293
Epoch 5  loss 11.807393074035645
Epoch 6  loss 12.415877342224121
Epoch 7  loss 11.098553657531738
Epoch 8  loss 8.564003944396973
Epoch 9  loss 5.84566068649292


[100.28304290771484,
 43.111087799072266,
 11.679686546325684,
 3.377065658569336,
 10.752118110656738,
 22.371599197387695,
 29.118043899536133,
 28.430830001831055,
 22.355804443359375,
 14.358283996582031]

know training the remote model number Two with the data for the ouwner number 2

In [44]:
remote_mode2= base_model.send(duet1)

In [45]:
remote_torch2 = duet1.torch 

In [47]:
params = remote_mode2.parameters()
optim2 = remote_torch2.optim.Adam(params=params , lr=0.1)

In [51]:
target2_ptr = th.FloatTensor(np.array([15,20,17,17,19]).reshape(-1,1))

In [52]:
target2_ptr

tensor([[15.],
        [20.],
        [17.],
        [17.],
        [19.]])

In [53]:
losses2 = train(iterations,remote_mode2,remote_torch2,optim2,data2_pointer,target2_ptr)

Epoch 0  loss 46.251644134521484
Epoch 1  loss 3.9336578845977783
Epoch 2  loss 27.339508056640625
Epoch 3  loss 26.661462783813477
Epoch 4  loss 10.607012748718262
Epoch 5  loss 3.5918662548065186
Epoch 6  loss 9.85547161102295
Epoch 7  loss 16.59503173828125
Epoch 8  loss 14.708015441894531
Epoch 9  loss 7.867440700531006


In [55]:
losses2

[46.251644134521484,
 3.9336578845977783,
 27.339508056640625,
 26.661462783813477,
 10.607012748718262,
 3.5918662548065186,
 9.85547161102295,
 16.59503173828125,
 14.708015441894531,
 7.867440700531006]

dans le cas réel on devait avoir un server qui fait l'aggregation des resultat entre le remote_model_1 et le remote_model_2 mais mtn on va esseyer de faire cela manuellement pour la simplicité 

on va esseyer de verifier les params du base model avec les 2  remote models 

In [56]:
params1 = remote_model1.parameters().get(request_block=True)
params2 = remote_mode2.parameters().get(request_block=True)

print("base model params :")
print(base_model.parameters())

print("remote model 1 params")
print(params1)

print("remote model 2 params")
print(params2)

base model params :
[Parameter containing:
tensor([[0.3345]], requires_grad=True), Parameter containing:
tensor([0.5879], requires_grad=True)]
remote model 1 params
[Parameter containing:
tensor([[-0.0038]], requires_grad=True), Parameter containing:
tensor([0.3196], requires_grad=True)]
remote model 2 params
[Parameter containing:
tensor([[0.2473]], requires_grad=True), Parameter containing:
tensor([0.5215], requires_grad=True)]


saving the updates


In [57]:
remote_model1_update = remote_model1.get(request_block=True).state_dict()

In [58]:
remote_mode2_update = remote_mode2.get(request_block=True).state_dict()

In [61]:
from collections import OrderedDict

In [62]:
avg_updates = OrderedDict()

In [64]:
avg_updates["linear.weight"] =( remote_mode2_update["linear.weight"] + remote_model1_update["linear.weight"]) /2
avg_updates["linear.bias"] = (remote_mode2_update["linear.bias"]+ remote_model1_update["linear.bias"])/2
avg_updates

OrderedDict([('linear.weight', tensor([[0.1218]])),
             ('linear.bias', tensor([0.4205]))])

mtn on a les weights aggréger donc on va les utiliser dans un modal combiné

In [65]:
combined_model = SyNet(torch)

In [66]:
combined_model.load_state_dict(avg_updates)

In [67]:
del avg_updates,remote_mode2_update,remote_model1_update

In [70]:
test_data = th.FloatTensor(np.array([5,15,95,75,80]).reshape(-1,1))
test_target = th.FloatTensor(np.array([3,4,3,17,19]).reshape(-1,1))

In [72]:
preds = []
with torch.no_grad(): 
    for i in range(len(test_data)): 
        sample = test_data[i]
        y_hat = combined_model(sample)
        print(f'prediction : {y_hat.item()}  la vrais valeur : {test_target[i]}')

prediction : 1.0294402837753296  la vrais valeur : tensor([3.])
prediction : 2.247253179550171  la vrais valeur : tensor([4.])
prediction : 11.98975658416748  la vrais valeur : tensor([3.])
prediction : 9.554130554199219  la vrais valeur : tensor([17.])
prediction : 10.163037300109863  la vrais valeur : tensor([19.])


d'apres les resultat on constate que le combined model nous a donné de bonne resultat

In [73]:
preds = []
with torch.no_grad(): 
    for i in range(len(test_data)): 
        sample = test_data[i]
        y_hat = base_model(sample)
        print(f'prediction : {y_hat.item()}  la vrais valeur : {test_target[i]}')

prediction : 2.260251045227051  la vrais valeur : tensor([3.])
prediction : 5.604864597320557  la vrais valeur : tensor([4.])
prediction : 32.36177062988281  la vrais valeur : tensor([3.])
prediction : 25.672544479370117  la vrais valeur : tensor([17.])
prediction : 27.344850540161133  la vrais valeur : tensor([19.])


on constate que le base model est plus fort que le base_model et cela est tout a fait nrml car il est trainer entre 2 dataset et non pas comme le base model